In [4]:
!pip -q install transformers peft accelerate bitsandbytes datasets sqlglot tqdm huggingface_hub

In [3]:
import os, json, re, sqlite3, random
from pathlib import Path
from typing import Dict
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
from tqdm import tqdm
import sqlglot


In [ ]:
import os
from huggingface_hub import login

login()  

Token has not been saved to git credential helper.


In [4]:
PREPARED_DIR = Path("./prepared") 
DATA_FILES = {
    "train": str(PREPARED_DIR/"spider_train.jsonl"),
    "dev":   str(PREPARED_DIR/"spider_dev.jsonl"),
}

ds = load_dataset("json", data_files=DATA_FILES)
ds


DatasetDict({
    train: Dataset({
        features: ['db_id', 'question', 'prompt', 'sql_gold'],
        num_rows: 7000
    })
    dev: Dataset({
        features: ['db_id', 'question', 'prompt', 'sql_gold'],
        num_rows: 1034
    })
})

In [18]:

BASE_MODEL = "meta-llama/Llama-3.2-3B"

print("Using:", BASE_MODEL)


Using: meta-llama/Llama-3.2-3B


In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

MAX_LEN = 2048  # drop to 1536 if you hit OOM


In [20]:
def map_to_features(ex):
    full   = ex["prompt"] + ex["sql_gold"]
    tok_f  = tokenizer(full, truncation=True, max_length=MAX_LEN)
    tok_p  = tokenizer(ex["prompt"], truncation=True, max_length=MAX_LEN)
    n_prompt = len(tok_p["input_ids"])

    labels = tok_f["input_ids"][:]
    labels[:n_prompt] = [-100]*n_prompt   # mask prompt tokens

    ex["input_ids"] = tok_f["input_ids"]
    ex["attention_mask"] = tok_f["attention_mask"]
    ex["labels"] = labels
    return ex

tok_ds = DatasetDict({
    "train": ds["train"].map(map_to_features, remove_columns=ds["train"].column_names),
    "dev":   ds["dev"].map(map_to_features,   remove_columns=ds["dev"].column_names),
})
tok_ds


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7000
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1034
    })
})

In [21]:
import torch, bitsandbytes as bnb
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

device_map = "auto"

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype="auto",
    load_in_4bit=True,          # QLoRA
    device_map=device_map,
    trust_remote_code=False
)

base = prepare_model_for_kbit_training(base)
peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)
model = get_peft_model(base, peft_cfg)
print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Trainable params: 24313856


In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=None,
    padding=True,
    label_pad_token_id=-100,
    return_tensors="pt",
)


In [24]:
from transformers import TrainingArguments

bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

args = TrainingArguments(
    output_dir="outputs/llama_spider_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,   # effective batch 16
    learning_rate=2e-4,
    num_train_epochs=3,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=25,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    bf16=bf16_ok,
    fp16=(torch.cuda.is_available() and not bf16_ok),
    report_to="none"
)


In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["dev"]
)

# fast sanity slice (adjust sizes as you like)
small_train = tok_ds["train"].shuffle(seed=42).select(range(3000))
small_dev   = tok_ds["dev"].shuffle(seed=42).select(range(500))
trainer.train_dataset = small_train
trainer.eval_dataset  = small_dev

trainer.train()

model.save_pretrained("outputs/llama_spider_lora/adapters")


/tmp/ipykernel_8667/1831611660.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,0.691100
50,0.294000
75,0.238500
100,0.243700
125,0.204000
150,0.203300
175,0.173600
200,0.138800
225,0.113000
250,0.114300


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


In [27]:
import torch

GEN_KW = dict(max_new_tokens=256, temperature=0.001, top_p=1.0, pad_token_id=tokenizer.eos_token_id)

@torch.inference_mode()
def generate_sql(prompt:str):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, **GEN_KW)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

def normalize_sql(s: str) -> str:
    try:
        return sqlglot.transpile(s, read="mysql", write="mysql", normalize=True, pretty=False)[0]
    except Exception:
        return s.strip()

def exact_match(pred: str, gold: str) -> bool:
    return normalize_sql(pred) == normalize_sql(gold)

subset = ds["dev"].select(range(100))
hits=0
for ex in tqdm(subset):
    pred = generate_sql(ex["prompt"])
    hits += exact_match(pred, ex["sql_gold"])
print("EM (subset):", hits/len(subset))


  0%|          | 0/100 [00:00<?, ?it/s]Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_values=None`.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 100/100 [47:19<00:00, 28.39s/it]

EM (subset): 0.0


In [28]:
save_dir = Path("outputs/llama_spider_lora/adapters")
save_dir.mkdir(parents=True, exist_ok=True)
model.save_pretrained(save_dir.as_posix())
print("Saved LoRA adapters to:", save_dir)

# Optional: push to HF Hub (adapters only)
# from huggingface_hub import create_repo
# create_repo("your-username/llama-spider-lora", private=True, exist_ok=True)
# model.push_to_hub("your-username/llama-spider-lora")
# tokenizer.push_to_hub("your-username/llama-spider-lora")


Saved LoRA adapters to: outputs/llama_spider_lora/adapters


In [7]:
from pathlib import Path
import torch, bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL  = "meta-llama/Llama-3.2-3B"        # EXACT model you trained on
ADAPTER_DIR = Path("outputs/llama_spider_lora/adapters")  # or checkpoint-564

# Tokenizer
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Base in 4-bit if CUDA; CPU fallback loads in full precision (slow)
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto" if torch.cuda.is_available() else None,
    load_in_4bit=True if torch.cuda.is_available() else False,
    torch_dtype="auto",
    trust_remote_code=False,
)

# 🔐 Attach LoRA adapters
model = PeftModel.from_pretrained(base, ADAPTER_DIR.as_posix())
model.eval()

# (Optional) merge for pure base weights inference (not needed now)
# model = model.merge_and_unload()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [8]:
# a) Check PEFT config/adapters
print("Active adapters:", getattr(model, "active_adapter", None))

# b) Make sure LoRA layers exist in the module names
has_lora = any("lora_A" in n or "lora_B" in n for n, _ in model.named_parameters())
print("Found LoRA params:", has_lora)

# c) Inspect one layer to see the injected LoRA
first_q = dict(model.named_modules()).get("base_model.model.model.layers.0.self_attn.q_proj", None)
print("First layer q_proj:", first_q)


Active adapters: default
Found LoRA params: True
First layer q_proj: lora.Linear4bit(
  (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=3072, out_features=16, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=16, out_features=3072, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)


In [10]:
prompt = "SYSTEM: Output ONLY SQL.\nUSER:\nQuestion: How many students are in the database?\nSchema:\nDatabase: academic\nTables:\n  student(id, *student_id, name)\n\nRules:\n- Only use tables/columns from academic.\n- Output only SQL, no commentary.\n"

inputs = tok(prompt, return_tensors="pt").to(model.device if torch.cuda.is_available() else "cpu")

with torch.inference_mode():
    out_base = base.generate(**inputs, max_new_tokens=64, temperature=0.001, top_p=1.0, pad_token_id=tok.eos_token_id)
    out_peft = model.generate(**inputs, max_new_tokens=64, temperature=0.001, top_p=1.0, pad_token_id=tok.eos_token_id)

print("\nBASE:", tok.decode(out_base[0], skip_special_tokens=True)[len(prompt):].strip()[:200])
print("PEFT:", tok.decode(out_peft[0], skip_special_tokens=True)[len(prompt):].strip()[:200])



BASE: SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM
PEFT: SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM student; SELECT count(*) FROM


In [12]:
# EVAL-1: paths + helpers (keeps your existing model/tok)
from pathlib import Path
import re, sqlite3, pandas as pd, sqlglot, torch
from datasets import load_dataset
from transformers import StoppingCriteria, StoppingCriteriaList

# ---- paths ----
PREPARED_DIR = Path("prepared")
DEV_JSONL    = PREPARED_DIR / "spider_dev.jsonl"
DB_ROOT      = Path("database")          # you already copied these to Lightning
OUT_CSV      = Path("outputs/dev_eval.csv")

# ---- (re)load dev if not in memory ----
if "ds" not in globals() or "dev" not in ds:
    ds = load_dataset("json", data_files={"dev": str(DEV_JSONL)})
dev_ds = ds["dev"]

# ---- decoding: stop exactly at first ';' ----
class StopAtSemicolon(StoppingCriteria):
    def __init__(self, tokenizer, prompt_len): self.tok = tokenizer; self.n = prompt_len
    def __call__(self, input_ids, scores, **kwargs):
        txt = self.tok.decode(input_ids[0][self.n:], skip_special_tokens=True)
        return ";" in txt

GEN = dict(
    max_new_tokens=256,
    temperature=0.001,
    top_p=1.0,
    pad_token_id=tok.eos_token_id,
    no_repeat_ngram_size=3,
    repetition_penalty=1.05
)

def robust_suffix(full: str, prompt: str) -> str:
    i = full.find(prompt)
    return full[i+len(prompt):].strip() if i != -1 else full.strip()

SQL_START = re.compile(r"(?is)\b(SELECT|WITH|INSERT|UPDATE|DELETE)\b")
SQL_SPAN  = re.compile(r"(?is)\b(SELECT|WITH|INSERT|UPDATE|DELETE)\b.*?;")

def extract_sql(text: str) -> str:
    m = SQL_SPAN.search(text)
    if m: return m.group(0).strip()
    m = SQL_START.search(text)
    return text[m.start():].strip() if m else text.strip()

@torch.inference_mode()
def generate_sql(prompt: str) -> str:
    inputs = tok(prompt, return_tensors="pt").to(model.device if torch.cuda.is_available() else "cpu")
    stopper = StoppingCriteriaList([StopAtSemicolon(tok, inputs["input_ids"].shape[1])])
    out = model.generate(**inputs, stopping_criteria=stopper, **GEN)
    full = tok.decode(out[0], skip_special_tokens=True)
    return extract_sql(robust_suffix(full, prompt))

def normalize_sql(s: str) -> str:
    try:
        return sqlglot.transpile(s, read="mysql", write="mysql", normalize=True, pretty=False)[0]
    except Exception:
        return s.strip()

def exact_match(pred: str, gold: str) -> bool:
    return normalize_sql(pred) == normalize_sql(gold)

def run_sql(db_id: str, sql: str):
    dbp = DB_ROOT / db_id / f"{db_id}.sqlite"
    if not dbp.exists(): return False, f"db_missing:{dbp}"
    try:
        con = sqlite3.connect(dbp.as_posix())
        df = pd.read_sql_query(sql, con); con.close()
        if len(df.columns)>0: df = df.sort_values(list(df.columns)).reset_index(drop=True)
        return True, df
    except Exception as e:
        return False, str(e)

def exec_match(pred_sql: str, gold_sql: str, db_id: str) -> bool:
    okp, rp = run_sql(db_id, pred_sql)
    okg, rg = run_sql(db_id, gold_sql)
    if not (okp and okg): return False
    try: return rp.equals(rg)
    except: return False


In [13]:
from tqdm import tqdm
import pandas as pd

EVAL_LIMIT = 100          # set to None for full dev
subset = dev_ds if EVAL_LIMIT is None else dev_ds.select(range(min(EVAL_LIMIT, len(dev_ds))))

rows = []
em_hits = ex_hits = 0
for ex in tqdm(subset, desc="Evaluating"):
    pred = generate_sql(ex["prompt"])
    em_ok = exact_match(pred, ex["sql_gold"])
    ex_ok = exec_match(pred, ex["sql_gold"], ex["db_id"])
    em_hits += int(em_ok); ex_hits += int(ex_ok)
    rows.append({
        "db_id": ex["db_id"],
        "question": ex["question"],
        "pred_sql": pred,
        "gold_sql": ex["sql_gold"],
        "em": int(em_ok),
        "ex": int(ex_ok),
    })

EM = em_hits / len(subset)
EX = ex_hits / len(subset)
print(f"\nEM: {EM:.3f} | EX: {EX:.3f} | N={len(subset)}")

df = pd.DataFrame(rows)
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)

# show a few misses for debugging
print("\nSample misses:")
for _, r in df[(df.em==0) | (df.ex==0)].head(5).iterrows():
    print("-"*80)
    print("DB:", r.db_id)
    print("Q :", r.question)
    print("P :", r.pred_sql)
    print("G :", r.gold_sql)


Evaluating: 100%|██████████| 100/100 [06:57<00:00,  4.17s/it]


EM: 0.300 | EX: 0.480 | N=100
Saved: outputs/dev_eval.csv

Sample misses:
--------------------------------------------------------------------------------
DB: concert_singer
Q : How many singers do we have?
P : SELECT count(*) FROM singer SELECT count(*),  name FROM singer GROUP BY name ORDER BY count(*) DESC LIMIT 1;
G : SELECT count(*) FROM singer
--------------------------------------------------------------------------------
DB: concert_singer
Q : How many singers are from each country?
P : SELECT count(*),  country FROM singer GROUP BY country;
G : SELECT country ,  count(*) FROM singer GROUP BY country
--------------------------------------------------------------------------------
DB: concert_singer
Q : List all song names by singers above the average age.
P : SELECT Song_Name FROM singer WHERE Age  >  (SELECT avg(Age) FROM singer) ORDER BY Age ASC;
G : SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)
-----------------------------------------------------